In [5]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import NLocal, CCXGate, CRZGate, RXGate, XGate

from qiskit_algorithms import optimizers
from qiskit.primitives import Estimator

from scipy.optimize import minimize

# import matplotlib.pyplot as plt

from qiskit import *
from qiskit.visualization import *
# from qiskit_aer import Aer

from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.properties import ParticleNumber
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper

# Para cargar lo necesario
import pickle

### Función para expandir el observable

In [6]:
def expand_hamiltonian(op, total_qubits):
    expanded_paulis = []
    for pauli, coeff in zip(op.paulis, op.coeffs):
        pauli_str = pauli.to_label()
        # Añadir identidades antes y después según la posición deseada
        new_pauli = (
            pauli_str + "I" * (total_qubits - len(pauli_str))
        )
        expanded_paulis.append((new_pauli, coeff))
    return SparsePauliOp.from_list(expanded_paulis)

### Función para crear el ansatz

In [7]:
def build_deep_ansatz(num_qubits):
    """Crea un circuito con L = 10n capas de ansatz para n qubits."""
    L = 10 * num_qubits  # número de capas
    qc = QuantumCircuit(num_qubits)
    thetas = []

    def layer(qc, theta_list):
        # RX en cada qubit
        for i in range(num_qubits):
            qc.rx(theta_list[i], i)
        # CZ entre qubits adyacentes
        for i in range(num_qubits - 1):
            qc.cz(i, i + 1)

    for layer_index in range(L):
        theta_layer = [Parameter(f'θ_{layer_index}_{i}') for i in range(num_qubits)]
        thetas.append(theta_layer)
        layer(qc, theta_layer)
        qc.barrier()

    return qc, thetas

In [12]:
# Cargar el Hamiltoniano desde el archivo
with open("hamiltonian_hidrogen_reduced.pkl", "rb") as f:
    base_hamiltonian= pickle.load(f)
print(base_hamiltonian)

SparsePauliOp(['II', 'ZI', 'IZ', 'ZZ', 'YY', 'XX'],
              coeffs=[-0.4804+0.j,  0.3435+0.j, -0.4347+0.j,  0.5716+0.j,  0.091 +0.j,
  0.091 +0.j])


In [23]:
max_qubits = 6

# Creamos el estimador para el simulador
estimator = Estimator()

for i in range(2, max_qubits):
    
    current_hamiltonian=expand_hamiltonian(base_hamiltonian, i)
    ansatz_circuit, thetas = build_deep_ansatz(i)

    # Parámetros iniciales
    x0 = np.zeros(len(thetas)*i)

    # Información sobre la iteración actual
    print(f"Preparando ejecución para {i} qubits.")
    print(f"Se usarán {len(thetas)*i} parámetros")

    # Diccionario para almacenar la evolución del costo
    cost_history_dict = {
        "prev_vector": None,
        "iters": 0,
        "cost_history": [],
    }

    def cost_func(params, ansatz, hamiltonian, estimator):
        job = estimator.run([ansatz], [hamiltonian], [params])
        result = job.result()
        energy = result.values[0]

        cost_history_dict["iters"] += 1
        cost_history_dict["prev_vector"] = params
        cost_history_dict["cost_history"].append(energy)

        print(f"Iteración {cost_history_dict['iters']}: Energía = {energy}")

        return energy

    # Ejecutamos la optimización
    res = minimize(
        cost_func,
        x0,
        args=(ansatz_circuit, current_hamiltonian, estimator),
        method="COBYLA",
    )


Preparando ejecución para 2 qubits.
Se usarán 40 parámetros
Iteración 1: Energía = 1.1102230246251565e-16
Iteración 2: Energía = -0.06293261432665165
Iteración 3: Energía = -0.29837565511745257
Iteración 4: Energía = -0.17280017108622356
Iteración 5: Energía = -0.9171088937137961
Iteración 6: Energía = -1.2132550960237063
Iteración 7: Energía = -1.3195596552399442
Iteración 8: Energía = -1.325160808439544
Iteración 9: Energía = -0.6454635921788181
Iteración 10: Energía = -1.3765335902265579
Iteración 11: Energía = -0.5934256392037097
Iteración 12: Energía = -1.4344989447040688
Iteración 13: Energía = -0.653098037556518
Iteración 14: Energía = -1.5240727740158078
Iteración 15: Energía = -0.8383860240348737
Iteración 16: Energía = -1.5682967172071303
Iteración 17: Energía = -1.1043477550086513
Iteración 18: Energía = -1.5838211837330132
Iteración 19: Energía = -1.4113450513666685
Iteración 20: Energía = -1.5238279977330573
Iteración 21: Energía = -1.279856872764034
Iteración 22: Energía 